In [1]:
# head off errors I was getting in Pairplot due to seaborn bug
!pip install --upgrade seaborn --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 6.4 MB/s eta 0:00:00a 0:00:01


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

from pathlib import Path

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# uncomment below if the runtime errors during the exploration and wrangling data get annoying
# import warnings
# warnings.filterwarnings("ignore", category=RuntimeWarning)


# imports for classification
from sklearn.tree import DecisionTreeClassifier

# Enforce Copy-on-Write
pd.set_option("mode.copy_on_write", True)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/surveilance/scholars_unders_surveillance_dataset_03-03-2021.csv
/kaggle/input/surveilance/Atlas of Surveillance-20251114.csv
/kaggle/input/surveilance/EFF-MuckRock 2016-2017 ALPR DATA.csv
/kaggle/input/surveilance/EFF-MuckRock 2016-2017 ALPR DATA - Definitions.csv
/kaggle/input/surveilance/aos-bordercounties_-_01.10.2020.csv
/kaggle/input/surveilance/EFF-MuckRock 2016-2017 ALPR DATA - Field Descriptions.csv


In [3]:
# set standard paths
cwd = Path().cwd() #set our cwd
project_folder = cwd.parent #set our project folder as the cwd parent

In [4]:
cwd

PosixPath('/kaggle/working')

In [5]:
project_folder

PosixPath('/kaggle')

In [6]:
# import data on types of surveilance used by law enforcement agencies - muliple states
aos_path = Path('input/surveilance/Atlas of Surveillance-20251114.csv') # using a data folder makes life easier
aos_file = project_folder / aos_path # join the project folder with the 
# dataset path
if not aos_file.exists():  # error catching in case the file is not found
	raise FileNotFoundError(f"Dataset file not found: {aos_file}")
# store file in a dataframe
df_aos = pd.read_csv(aos_file)

In [7]:
# import data from Electronic Frontier Foundation on use of Automated License Plate Readers - multiple states
alpr_data_path = Path('input/surveilance/EFF-MuckRock 2016-2017 ALPR DATA.csv') # using a data folder makes life easier
alpr_data_file = project_folder / alpr_data_path # join the project folder with the 
# dataset path
if not alpr_data_file.exists():  # error catching in case the file is not found
	raise FileNotFoundError(f"Dataset file not found: {alpr_data_file}")
# store file in a dataframe
df_alpr_data = pd.read_csv(alpr_data_file)
    

In [8]:
# import reference data from Electronic Frontier Foundation on use of Automated License Plate Readers - multiple states
# definitions of terms used
alpr_defs_path = Path('input/surveilance/EFF-MuckRock 2016-2017 ALPR DATA - Definitions.csv') # using a data folder makes life easier
alpr_defs_file = project_folder / alpr_defs_path # join the project folder with the 
# dataset path
if not alpr_defs_file.exists():  # error catching in case the file is not found
	raise FileNotFoundError(f"Dataset file not found: {alpr_defs_file}")
# store file in a dataframe
df_alpr_defs = pd.read_csv(alpr_defs_file)

# field descriptions
alpr_desc_path = Path('input/surveilance/EFF-MuckRock 2016-2017 ALPR DATA - Field Descriptions.csv') # using a data folder makes life easier
alpr_desc_file = project_folder / alpr_desc_path # join the project folder with the 
# dataset path
if not alpr_desc_file.exists():  # error catching in case the file is not found
	raise FileNotFoundError(f"Dataset file not found: {alpr_desc_file}")
# store file in a dataframe
df_alpr_desc = pd.read_csv(alpr_desc_file)

In [9]:
# import reference data from Electronic Frontier Foundation inventory on types of surveilance technologies used by border states
aos_brdr_path = Path('input/surveilance/aos-bordercounties_-_01.10.2020.csv') # using a data folder makes life easier
aos_brdr_file = project_folder / aos_brdr_path # join the project folder with the 
# dataset path
if not aos_brdr_file.exists():  # error catching in case the file is not found
	raise FileNotFoundError(f"Dataset file not found: {aos_brdr_file}")
# store file in a dataframe
df_aos_brdr = pd.read_csv(aos_brdr_file)

In [10]:
# import reference data from Electronic Frontier Foundation on types of surveilance used by campus police in colleges- multiple states
schol_srv_path = Path('input/surveilance/scholars_unders_surveillance_dataset_03-03-2021.csv') # using a data folder makes life easier
schol_srv_file = project_folder / schol_srv_path # join the project folder with the 
# dataset path
if not schol_srv_file.exists():  # error catching in case the file is not found
	raise FileNotFoundError(f"Dataset file not found: {schol_srv_file}")
# store file in a dataframe
df_schol_srv = pd.read_csv(schol_srv_file)

# Explore Data / Data Wrangling

import data from Electronic Frontier Foundation on use of Automated License Plate Readers - multiple statesFirst thing - opened files in Excel and did some clean up.  Filtered each column and scrolled through the unique/ filterable values Excel found.  Filled all blank values with an empty string to avoid NaN errors.  Changed full state name to two-letter state abbreviation for consistency across all State columns.  Removed invalid characters, spaces from column headers, changing them to PascalCase instead.
AOS Data had some blank entries for counties.  Looked them up using Google. Entered the Census area as county for AK cities that do not have an actual county. 
Entered DC Metro Area as county for Washington DC
Entered CityName (no county) for independent US cities not existing in a county (ex: St Louis, MO, Baltimore, MD)
Separated multiple counties listed in one cell to multiple cells county1, county2..
Checking for typos/ misspelled counties/ invalid characters



*Unintended bonus of uploading the data to kaggle- it provides preview tables and does some analysis on its own!*

df_alpr_desc and df_alpr_defs are small dataframes that just supply reference information form the values and columns for the df_alpr_data database.
Display for reference

In [11]:
df_alpr_desc

,Agency,State,DirectSharing,NVLS,2016Detections,2016Hits,2017Detections,2017Hits,CombinedDetections,CombinedHits,AllTimeDetections,AllTimeHits,DataSharingLink,2016HitRatioReport,2017Hit RatioReport,2016DetectionReport,2016HitReport,2017DetectionReport,2017HitReport,CombinedDetectionHit Data
0,Name of agency,State where agency is located,Number of agencies listed on the Data Sharing ...,National Vehicle Location Service (sometimes r...,Total number of license plate scans (detection...,Total number of times in 2016 a scanned licens...,Number of license plate scans (detections) col...,Total number of times in 2017 a scanned licens...,"A * indicates incomplete data, often because t...","A * indicates incomplete data, often because t...",This field only includes agencies that provide...,This field only includes agencies that provide...,Link to the Data Sharing Report or similar rec...,"Link to the 2016 ""Hit Ratio Report"" or similar...","Link to the 2017 ""Hit Ratio Report"" or similar...","Link to the 2016 ""Detection Report"" or similar...","Link to the 2016 ""Hit Report"" or similar recor...","Link to the 2017 ""Detection Report"" or similar...","Link to the 2017 ""Hit Report"" or similar recor...",Link to documents that contain data for both d...
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Note: In most cases this figure was generated ...,Note: In most cases this figure was generated ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df_alpr_defs

,Definitions,Unnamed: 1
0,NaN,Not applicable - refers to documents that woul...
1,Not Provided,Refers to when an agency failed to provided re...
2,Data Incomplete,Refers to when an agency provided incomplete d...


Take a peek at each dataset

In [13]:
df_aos.head()

,AOSNUMBER,City,Counties,County1,County2,County3,County4,County5,County6,County7,...,Link2Source,Link2Type,Link2Date,Link2DateComment,Link3,Link3Snapshot,Link3Source,Link3Type,Link3Date,OtherLinks
0,AOS002987,Anchorage,Anchorage Borough,Anchorage Borough,NaN,NaN,NaN,NaN,NaN,NaN,...,Center for the Study of the Drone at Bard College,,7/12/1905,,https://www.muni.org/Departments/Assembly/Docu...,,www.muni.org,,,
1,AOS004259,Anchorage,Anchorage Borough,Anchorage Borough,NaN,NaN,NaN,NaN,NaN,NaN,...,Electronic Frontier Foundation,,4/7/2014,,https://www.openthegovernment.org/dhs-fusion-c...,,Open the Government,,4/2/2020,
2,AOS006868,Anchorage,Anchorage Borough,Anchorage Borough,NaN,NaN,NaN,NaN,NaN,NaN,...,,,,,,,,,,
3,AOS007456,Anchorage,Anchorage Borough,Anchorage Borough,NaN,NaN,NaN,NaN,NaN,NaN,...,Municipality of Anchorage,,7/21/2009,,https://www.ktuu.com/content/news/Anchorage-po...,,2KTUU,,8/18/2016,
4,AOS013830,Anchorage,Anchorage Borough,Anchorage Borough,NaN,NaN,NaN,NaN,NaN,NaN,...,Alaska's News Source,,8/24/2023,,https://www.anchoragepolice.com/news/body-worn...,,Anchorage Police Department Website,,10/3/2023,


In [14]:
df_schol_srv.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,Ref,Agency,City,County,State,TypeofLEA,TypeofJuris,Technology,Vendor,Summary,...,Link2Source,Link2Date,Link2DateComment,Link2Type,Link3,Link3Source,Link3Date,Link3DateComment,Link3Type,OtherLinks
0,EDU001,Abraham Baldwin Agricultural College Police De...,Tifton,Tift County,GA,Police,University,Body-worn Cameras,NaN,The Abraham Baldwin Agricultural College Polic...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EDU002,Agnes Scott College Department of Public Safety,Decatur,DeKalb County,GA,Police,University,Body-worn Cameras,Axon,Police officers with the Agnes Scott College D...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,EDU003,Albany State University Police Department,Albany,Dougherty County,GA,Police,University,Body-worn Cameras,NaN,The Albany State University Police Department ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,EDU004,Arizona State University Police Department,Tempe,Maricopa County,AZ,Police,University,Body-worn Cameras,NaN,The Arizona State University Police Department...,...,ASU Police Department Facebook page,1/31/2020,NaN,Social media,NaN,NaN,NaN,NaN,NaN,NaN
4,EDU005,Arkansas State University Police Department,Jonesboro,Craighead County,AR,Police,University,Body-worn Cameras,NaN,The Arkansas State University Police Departmen...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df_aos.dtypes

AOSNUMBER            object
City                 object
Counties             object
County1              object
County2              object
County3              object
County4              object
County5              object
County6              object
County7              object
State                object
Agency               object
TypeOfLEA            object
Summary              object
TypeOfJuris          object
Technology           object
Vendor               object
Link1                object
Link1Snapshot       float64
Link1Source          object
Link1Type            object
Link1Date            object
Link1DateComment     object
Link 2               object
Link2Snapshot        object
Link2Source          object
Link2Type            object
Link2Date            object
Link2DateComment     object
Link3                object
Link3Snapshot        object
Link3Source          object
Link3Type            object
Link3Date            object
OtherLinks           object
dtype: object

In [16]:
# to get all rows to show, force pandas not to redact
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

pd.DataFrame({
    'aosDtypes': df_aos.dtypes,
    'alprDtypes': df_alpr_data.dtypes,
    'borderDtypes': df_aos_brdr.dtypes,
    'collegeDtypes': df_schol_srv.dtypes
})

# list is really long, not sure if that was really useful after all

,aosDtypes,alprDtypes,borderDtypes,collegeDtypes
2016DetectionReport,NaN,object,NaN,NaN
2016Detections,NaN,object,NaN,NaN
2016HitRatioReport,NaN,object,NaN,NaN
2016HitReport,NaN,object,NaN,NaN
2016Hits,NaN,object,NaN,NaN
2016To2017Detections,NaN,object,NaN,NaN
2016To2017Hits,NaN,object,NaN,NaN
2017DetectionReport,NaN,object,NaN,NaN
2017Detections,NaN,object,NaN,NaN
2017HitRatioReport,NaN,object,NaN,NaN


In [17]:
df_aos.columns

Index(['AOSNUMBER', 'City', 'Counties', 'County1', 'County2', 'County3',
       'County4', 'County5', 'County6', 'County7', 'State', 'Agency',
       'TypeOfLEA', 'Summary', 'TypeOfJuris', 'Technology', 'Vendor', 'Link1',
       'Link1Snapshot', 'Link1Source', 'Link1Type', 'Link1Date',
       'Link1DateComment', 'Link 2', 'Link2Snapshot', 'Link2Source',
       'Link2Type', 'Link2Date', 'Link2DateComment', 'Link3', 'Link3Snapshot',
       'Link3Source', 'Link3Type', 'Link3Date', 'OtherLinks'],
      dtype='object')

In [18]:
df_alpr_data.head()

,Agency,State,DirectSharing,NVLS,2016Detections,2016Hits,2017Detections,2017Hits,2016To2017Detections,2016To2017Hits,AllTimeDetections,AllTimeHits,DataSharingLink,2016HitRatioReport,2017HitRatioReport,2016DetectionReport,2016HitReport,2017DetectionReport,2017HitReport,CombinedDetectionHit Data
0,Acworth Police Department,GA,270,Y,Not Provided,Not Provided,Not Provided,Not Provided,Not Provided,Not Provided,NaN,NaN,https://www.documentcloud.org/documents/4945894-Data-Sharing-Report-Acworth-Police-Department.html,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alamo Heights Police Department,TX,14,Y,"1,034",Not Provided,"407,825",Not Provided,"408,859",0,NaN,NaN,https://www.documentcloud.org/documents/4501639-Data-Sharing-Report-Alamo-Heights-Police.html,https://www.documentcloud.org/documents/4559445-Alamo-Heights-Police-Department-Detection-and.html,https://www.documentcloud.org/documents/4559446-Alamo-Heights-Police-Department-Detection-and.html,NaN,NaN,NaN,NaN,NaN
2,American Canyon Police Department,CA,27,Y,"786,289",271,"638,034",194,"1,424,323",465,NaN,NaN,https://www.documentcloud.org/documents/4501640-Data-Sharing-Report-American-Canyon-Police.html,https://www.documentcloud.org/documents/4559447-American-Canyon-Police-Department-Detection-and.html,https://www.documentcloud.org/documents/4559448-American-Canyon-Police-Department-Detection-and.html,NaN,NaN,NaN,NaN,NaN
3,Atlanta Police Department,GA,8,NaN,Not Provided,Not Provided,"128,555,383","786,580","128,555,383*","786,580*",NaN,NaN,https://www.documentcloud.org/documents/4501641-Data-Sharing-Report-Atlanta-Police-Department.html,NaN,NaN,NaN,NaN,NaN,NaN,https://www.documentcloud.org/documents/4559449-Atlanta-Police-Department-All-Info.html
4,Auburn Police Department,CA,68,Y,Not Provided,Not Provided,Not Provided,Not Provided,Not Provided,Not Provided,NaN,NaN,https://www.documentcloud.org/documents/4802024-Data-Sharing-Report-Auburn-Police-Department.html,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Which states are represented in each dataset?

In [19]:
aos_states = np.sort(df_aos['State'].unique())
alpr_states = np.sort(df_alpr_data['State'].unique())
brdr_states = np.sort(df_aos_brdr['State'].unique())
schol_states = np.sort(df_schol_srv['State'].unique())

print("aos states: ", aos_states, "\n\nalpr states: ", alpr_states, "\n\nborder states: ", brdr_states, "\n\ncollege states: ", schol_states)

aos states:  ['AK' 'AL' 'AR' 'AZ' 'CA' 'CO' 'CT' 'DC' 'DE' 'FL' 'GA' 'GU' 'HI' 'IA'
 'ID' 'IL' 'IN' 'KS' 'KY' 'LA' 'MA' 'MD' 'ME' 'MI' 'MN' 'MO' 'MS' 'MT'
 'NC' 'ND' 'NE' 'NH' 'NJ' 'NM' 'NV' 'NY' 'OH' 'OK' 'OR' 'PA' 'PR' 'RI'
 'SC' 'SD' 'TN' 'TX' 'UT' 'VA' 'VI' 'VT' 'WA' 'WI' 'WV' 'WY'] 

alpr states:  ['AL' 'AZ' 'CA' 'CO' 'CT' 'FL' 'GA' 'IA' 'IL' 'IN' 'KS' 'LA' 'MD' 'MI'
 'MN' 'MO' 'NJ' 'NY' 'OH' 'OR' 'PA' 'TX' 'US' 'WA'] 

border states:  ['AZ' 'CA' 'NM' 'TX'] 

college states:  ['AL' 'AR' 'AZ' 'CA' 'CO' 'CT' 'FL' 'GA' 'IA' 'IL' 'IN' 'KS' 'KY' 'LA'
 'MA' 'MD' 'MI' 'MO' 'MS' 'NC' 'ND' 'NE' 'NJ' 'NM' 'NV' 'NY' 'OH' 'OR'
 'PA' 'SC' 'SD' 'TN' 'TX' 'UT' 'VA' 'WI' 'WY']


It looks like the border states dataset has the fewest states. That makes sense since it consists of only southern border states.
Do all of the other databases contain those four states? 

In [20]:
# states in the border state data also in the alpr data
print("border & alpr " ,np.sort(df_alpr_data[df_alpr_data['State'].isin(brdr_states)]['State'].unique()))
# states in the border state data also in the aos data
print("border & aos ", np.sort(df_aos[df_aos['State'].isin(brdr_states)]['State'].unique()))
# states in the border state data also in the college data
print("border & college ", np.sort(df_schol_srv[df_schol_srv['State'].isin(brdr_states)]['State'].unique()))

border & alpr  ['AZ' 'CA' 'TX']
border & aos  ['AZ' 'CA' 'NM' 'TX']
border & college  ['AZ' 'CA' 'NM' 'TX']


So at this point, do we narrow down the states we use to only those in the border state data, or do we drop the border state dataset and do analysis on the other 3 datasets?
How much data does each method leave us with?

I just want to look at the column headers next to eachother so I can compare and see what data we have across multple dfs

In [22]:
df_aos.head(0)

,AOSNUMBER,City,Counties,County1,County2,County3,County4,County5,County6,County7,State,Agency,TypeOfLEA,Summary,TypeOfJuris,Technology,Vendor,Link1,Link1Snapshot,Link1Source,Link1Type,Link1Date,Link1DateComment,Link 2,Link2Snapshot,Link2Source,Link2Type,Link2Date,Link2DateComment,Link3,Link3Snapshot,Link3Source,Link3Type,Link3Date,OtherLinks


In [177]:
print(df_aos['State'].unique())
len(df_aos['State'].unique())

['AK' 'AL' 'AR' 'AZ' 'CA' 'CO' 'CT' 'DC' 'DE' 'FL' 'GA' 'GU' 'HI' 'IA'
 'ID' 'IL' 'IN' 'KS' 'KY' 'LA' 'MA' 'MD' 'ME' 'MI' 'MN' 'MO' 'MS' 'MT'
 'NE' 'NC' 'ND' 'NH' 'NJ' 'NM' 'NV' 'NY' 'OH' 'OK' 'OR' 'PA' 'PR' 'RI'
 'SC' 'SD' 'TN' 'TX' 'UT' 'VA' 'VI' 'VT' 'WA' 'WI' 'WV' 'WY']


54

In [23]:
df_aos_brdr.head(0)

,RefNo,Agency,City,County,State,LEAType,JurisdictionType,TechnologyType,Vendor,Summary,PrimaryLink,ArchivedPrimaryLink,DocumentType,Source,DocumentDate,Unnamed: 15,AdditionalLink1,AdditionalLink2,AdditionalLink3,ArchivedAdditionalLink1,AdditionalArchivedLink2,AdditionalArchivedLink3,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28


In [24]:
df_schol_srv.head(0)

,Ref,Agency,City,County,State,TypeofLEA,TypeofJuris,Technology,Vendor,Summary,Link1,Link1Source,Link1Date,Link1DateComment,Link1Type,Link2,Link2Source,Link2Date,Link2DateComment,Link2Type,Link3,Link3Source,Link3Date,Link3DateComment,Link3Type,OtherLinks


In [168]:
df_alpr_data.head(0)

,Agency,State,DirectSharing,NVLS,2016Detections,2016Hits,2017Detections,2017Hits,2016To2017Detections,2016To2017Hits,AllTimeDetections,AllTimeHits,DataSharingLink,2016HitRatioReport,2017HitRatioReport,2016DetectionReport,2016HitReport,2017DetectionReport,2017HitReport,CombinedDetectionHit Data


Do the other dataframes reference ALPR as a method of surveilance used? 

In [25]:
print("AOS ", df_aos['Technology'].value_counts())
print("\nAOS border ",df_aos_brdr['TechnologyType'].value_counts())
print("\nCollege ",df_schol_srv['Technology'].value_counts())

AOS  Technology
Body-worn Cameras                      5334
Automated License Plate Readers        4101
Drones                                 1772
Third-party Investigative Platforms    1104
Face Recognition                        987
Camera Registry                         770
Gunshot Detection                       258
Real-Time Crime Center                  242
Predictive Policing                     206
Cell-site Simulator                      91
Fusion Center                            90
Video Analytics                          90
Name: count, dtype: int64

AOS border  TechnologyType
Body-worn Cameras                                       50
Automated License Plate Readers                         34
Drones                                                  23
Camera Network                                          14
Mobile Surveillance Vehicle                             14
Spy Plane                                                7
Remote Video Surveillance Systems               

Automated License plate readers appear to make up a significant amount of the data, so it should still be considered.

Since the AOS border county data is provided from the same source and similar heading of the AOS dataset, is it acutally a subset? 

What counties are in the border counties data?

In [26]:
brdr_counties = df_aos_brdr['County'].unique()
brdr_counties

array(['Cochise County', 'Maricopa County ', 'Pima County',
       'Pima County ', 'Santa Cruz County', 'Yuma County',
       'Imperial County', 'San Diego County', 'San Diego County ',
       'Dona Ana County', 'Luna County', 'Brewster County',
       'Cameron County', 'El Paso County', 'El Paso County ',
       'Hidalgo County', 'Hidalgo County ', 'Hudspeth County',
       'Jeff Davis County', 'Kinney County', 'Maverick County',
       'Presidio County', 'Starr County', 'Terrell County',
       'Travis County', 'Val Verde County', 'Webb County',
       'Zapata County'], dtype=object)

Some of the counties are duplicated because of trailing spaces.  Let's trim them. 

In [78]:
df_aos_brdr['County'] = df_aos_brdr['County'].str.strip()

In [79]:
brdr_counties = df_aos_brdr['County'].unique()
brdr_counties

array(['Cochise County', 'Maricopa County', 'Pima County',
       'Santa Cruz County', 'Yuma County', 'Imperial County',
       'San Diego County', 'Dona Ana County', 'Luna County',
       'Brewster County', 'Cameron County', 'El Paso County',
       'Hidalgo County', 'Hudspeth County', 'Jeff Davis County',
       'Kinney County', 'Maverick County', 'Presidio County',
       'Starr County', 'Terrell County', 'Travis County',
       'Val Verde County', 'Webb County', 'Zapata County'], dtype=object)

Duplicates are gone now, and leading and trailing spaces are gone.  

Do the counties in the AOS df also have this problem? Let's take a quick look at the Texas Counties, since we saw a couple in the brdr data that did. 

In [103]:
df_aos[df_aos['State']=='TX']['Counties'].unique()
len(df_aos[df_aos['State']=='TX']['Counties'].unique())

170

It does not appear that these have trailing spaces.  

In [88]:
df_aos['Counties'] = df_aos['Counties'].str.strip()

In [102]:
#Lets look again and see if anything changed
df_aos[df_aos['State']=='TX']['Counties'].unique()
len(df_aos[df_aos['State']=='TX']['Counties'].unique())


170

I had to split list in "Counties" to multiple county columns, County1, County2, up to County7.  In order to compare I want to see what is the max# of County rows I need to include.
Using value_counts() instead of unique() for this will give me a list of the values so I can see which value returned has the longest list of Counties, so then I will know how many County columns to include in comparison. 

In [27]:
df_aos[df_aos['State'].isin(brdr_states)]['Counties'].value_counts()

Counties
Los Angeles County                                                    199
Maricopa County                                                        77
Orange County                                                          74
Contra Costa County                                                    73
San Diego County                                                       65
Alameda County                                                         57
Tarrant County                                                         57
Harris County                                                          57
Dallas County                                                          56
Riverside County                                                       52
Santa Clara County                                                     45
San Bernardino County                                                  36
San Mateo County                                                       32
Travis County                

The aos data contains more than just the border counties.  That makes sense.  Look at the aos data for just the state and counties included in the border counties data.

We can also easily see that the longest list returned from the "Counties" column of the aos dataset is 4.  So when comparing, I need to include the columns for County1 through County4 of the aos data to see if the border counties are included.

In [104]:
# df_aos[(df_aos['State'].isin(brdr_states)) & 
#brdr_cnty_mask = (df_aos['County1'].isin(brdr_counties)) | (df_aos['County2'].isin(brdr_counties)) | (df_aos['County3'].isin(brdr_counties)) | (df_aos['County4'].isin(brdr_counties)) #]
# this logic is long, but I can't think of a more concise but also clear way to do it
df_aos_ctycontymatch_brdr = df_aos[(df_aos['State'].isin(brdr_states)) & (df_aos['County1'].isin(brdr_counties)) | (df_aos['County2'].isin(brdr_counties)) | (df_aos['County3'].isin(brdr_counties)) | (df_aos['County4'].isin(brdr_counties))]
df_aos_ctycontymatch_brdr.head(1)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,AOSNUMBER,City,Counties,County1,County2,County3,County4,County5,County6,County7,State,Agency,TypeOfLEA,Summary,TypeOfJuris,Technology,Vendor,Link1,Link1Snapshot,Link1Source,Link1Type,Link1Date,Link1DateComment,Link 2,Link2Snapshot,Link2Source,Link2Type,Link2Date,Link2DateComment,Link3,Link3Snapshot,Link3Source,Link3Type,Link3Date,OtherLinks
382,AOS000692,Bisbee,Cochise County,Cochise County,NaN,NaN,NaN,NaN,NaN,NaN,AZ,Cochise County Sheriff's Office,Sheriff,"The Cochise County Sheriff's office received approval in 2019 to spend $580,784 over five years to outfit 60 officers with Axon body-worn cameras.",County,Body-worn Cameras,Axon,http://agenda.cochise.az.gov/agenda_publish.cfm?id=&mt=ALL&get_month=3&get_year=2019&dsp=agm&seq=4366&rev=0&ag=1606&ln=56841&nseq=&nrev=&pseq=&prev=#ReturnTo56841,NaN,Cochise Website,,3/26/2019,,,,,,,,,,,,,


In [105]:
df_aos_brdr.head(1)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,RefNo,Agency,City,County,State,LEAType,JurisdictionType,TechnologyType,Vendor,Summary,PrimaryLink,ArchivedPrimaryLink,DocumentType,Source,DocumentDate,Unnamed: 15,AdditionalLink1,AdditionalLink2,AdditionalLink3,ArchivedAdditionalLink1,AdditionalArchivedLink2,AdditionalArchivedLink3,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28
0,AZCOC010,Cochise County Sheriff's Office,Bisbee,Cochise County,AZ,Sheriff,County,Body-worn Cameras,Axon,"The Cochise County Sheriff's office received approve to spend $580,784 over five years to outfit 60 officers with Axon body-worn cameras.",http://agenda.cochise.az.gov/agenda_publish.cfm?id=&mt=ALL&get_month=3&get_year=2019&dsp=agm&seq=4366&rev=0&ag=1606&ln=56841&nseq=&nrev=&pseq=&prev=#ReturnTo56841,http://web.archive.org/web/20190820224316/http://agenda.cochise.az.gov/agenda_publish.cfm?id=&mt=ALL&get_month=3&get_year=2019&dsp=agm&seq=4366&rev=0&ag=1606&ln=56841&nseq=&nrev=&pseq=&prev=,Info on a government website,Cochise Website,3/26/2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Which columns do the two dataframes have in common?

In [106]:
df_aos_ctycontymatch_brdr.head(0)

,AOSNUMBER,City,Counties,County1,County2,County3,County4,County5,County6,County7,State,Agency,TypeOfLEA,Summary,TypeOfJuris,Technology,Vendor,Link1,Link1Snapshot,Link1Source,Link1Type,Link1Date,Link1DateComment,Link 2,Link2Snapshot,Link2Source,Link2Type,Link2Date,Link2DateComment,Link3,Link3Snapshot,Link3Source,Link3Type,Link3Date,OtherLinks


In [107]:
df_aos_brdr.head(0)

,RefNo,Agency,City,County,State,LEAType,JurisdictionType,TechnologyType,Vendor,Summary,PrimaryLink,ArchivedPrimaryLink,DocumentType,Source,DocumentDate,Unnamed: 15,AdditionalLink1,AdditionalLink2,AdditionalLink3,ArchivedAdditionalLink1,AdditionalArchivedLink2,AdditionalArchivedLink3,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28


City, Border County will be in AOS County1 - County4, Agency=Agency, AOS TypeOfLEA = brdr LEAType   AOS TypeOfJuris = brdr JurisdictionType AOS Technology = brdr TechnologyType  AOS Vendor = brder Vendor AOS Summary = brder Summary

All the rest is descriptive text will not be useful for what we are comparing right now

In [108]:
aos_cols = ['County1', 'County2', 'County3', 'County4', 'State', 'Agency', 'TypeOfLEA', 'TypeOfJuris', 'Technology', 'Vendor' ,'Summary']
brdr_cols = ['County', 'State', 'Agency', 'LEAType', 'JurisdictionType',  'TechnologyType', 'Vendor', 'Summary' ]

In [109]:
df_aos_sm = df_aos_ctycontymatch_brdr[aos_cols]
df_aos_sm.head(1)

,County1,County2,County3,County4,State,Agency,TypeOfLEA,TypeOfJuris,Technology,Vendor,Summary
382,Cochise County,NaN,NaN,NaN,AZ,Cochise County Sheriff's Office,Sheriff,County,Body-worn Cameras,Axon,"The Cochise County Sheriff's office received approval in 2019 to spend $580,784 over five years to outfit 60 officers with Axon body-worn cameras."


In [110]:
df_aos_brdr_sm = df_aos_brdr[brdr_cols]
df_aos_brdr_sm.head(1)

,County,State,Agency,LEAType,JurisdictionType,TechnologyType,Vendor,Summary
0,Cochise County,AZ,Cochise County Sheriff's Office,Sheriff,County,Body-worn Cameras,Axon,"The Cochise County Sheriff's office received approve to spend $580,784 over five years to outfit 60 officers with Axon body-worn cameras."


In [111]:
df_aos_sm.info()

<class 'pandas.core.frame.DataFrame'>
Index: 339 entries, 382 to 13791
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   County1      339 non-null    object
 1   County2      6 non-null      object
 2   County3      0 non-null      object
 3   County4      0 non-null      object
 4   State        339 non-null    object
 5   Agency       339 non-null    object
 6   TypeOfLEA    339 non-null    object
 7   TypeOfJuris  339 non-null    object
 8   Technology   339 non-null    object
 9   Vendor       193 non-null    object
 10  Summary      339 non-null    object
dtypes: object(11)
memory usage: 31.8+ KB


It doesnt look like any of our matching counties were in a County2+ position, so let's go ahead and drop these from our small df and then rename the column County to match the border df.  We are ok to drop, because this is just the copy. 

In [129]:
df_aos_sm = df_aos_sm.drop(['County2', 'County3', 'County4'], axis=1)

In [132]:
df_aos_sm = df_aos_sm.rename(columns = {'County1' : 'County'})
df_aos_sm.head(1)

,County,State,Agency,TypeOfLEA,TypeOfJuris,Technology,Vendor,Summary
382,Cochise County,AZ,Cochise County Sheriff's Office,Sheriff,County,Body-worn Cameras,Axon,"The Cochise County Sheriff's office received approval in 2019 to spend $580,784 over five years to outfit 60 officers with Axon body-worn cameras."


In [133]:
df_aos_brdr_sm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207 entries, 0 to 206
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   County            207 non-null    object
 1   State             207 non-null    object
 2   Agency            207 non-null    object
 3   LEAType           207 non-null    object
 4   JurisdictionType  207 non-null    object
 5   TechnologyType    207 non-null    object
 6   Vendor            89 non-null     object
 7   Summary           207 non-null    object
dtypes: object(8)
memory usage: 13.1+ KB


Looking at the summary sections in the data viewer, they are the most unique.  Let's take a look at them for one county for each and see how similar or different they are.  (Because they are many sentences of text, I am not sure they can be accuratley compared with == or .isequal unless they are EXACTLY the same and truly just duplicated data.  Checking feasibility of a comparison with one of these methods. 

For this quick check, I would like to look at a county with a small number of records to see if they are duplicates. 

In [135]:
df_aos_brdr_sm['County'].value_counts()

County
San Diego County     46
Hidalgo County       26
Pima County          17
El Paso County       17
Webb County          15
Cochise County       13
Cameron County       11
Imperial County      10
Dona Ana County       8
Yuma County           8
Starr County          7
Maverick County       6
Val Verde County      5
Presidio County       3
Zapata County         3
Maricopa County       2
Brewster County       2
Travis County         2
Jeff Davis County     1
Hudspeth County       1
Terrell County        1
Luna County           1
Santa Cruz County     1
Kinney County         1
Name: count, dtype: int64

Let's try Val Verde County.  It has 5 records in the border dataset, that give us a decent chance of having a duplicate without having to review too much data

Now let's compare to the aos data.

There are some duplicate summaries. Let's see if the whole record is the same. brd 185 = aos 13725 brd 186 = 13726 